# Co-localization Analysis on Drosophila spreads

This notebook analyses co-localization on Drosophila salivary glands spreads

### Import Packages

In [1]:
import warnings

import numpy as np

warnings.filterwarnings('ignore')

# Import Cell Profiler Dependencies
import cellprofiler_core.preferences as cpprefs
import cellprofiler_core.pipeline as cpp
cpprefs.set_headless()
from cellprofiler_core.modules.injectimage import InjectImage

# import ezomero
import ezomero

# Import Python System Packages
import os
import tempfile
import pandas
from getpass import getpass

### Set Cell Output Directory

In [2]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials and dataset to analyze

In [18]:
print("Provide the credentials. If you are connected as a guest in read-only mode use 'public' as login and password")
conn = ezomero.connect(user=input("OMERO Username: "),
                       password=getpass("OMERO Password: "),
                       group=input("OMERO Group (leave empty if guest user): ") or None,
                       host='bioimage.france-bioinformatique.fr',
                       port=4072,
                       secure=True)

print(f"Successful connection: {conn.isConnected()}")

Provide the credentials. If you are connected as a guest in read-only mode use 'public' as login and password
Successful connection: True


If you saw a "True" here, congratualtions, you are connected to the OMERO server

### Load a cellprofiler pipeline as part of the shared work flow
We are loading the Cellprofiler analysis pipe to use it within this notebook.
Instead of using CellProfiler's interface to load the images we have to remove the first 4 modules of the pipeline that are in charge of loading the images

In [5]:
pipeline = cpp.Pipeline()
pipeline.load("../cell_profiler_pipelines/Coloc_v5.cppipe")

# Remove first 4 modules: Images, Metadata, NamesAndTypes, Groups...
# (replaced by InjectImage module below)
for i in range(4):
    print('Remove module: ', pipeline.modules()[0].module_name)
    pipeline.remove_module(1)
for i in range(4):
    print('Remove module: ', pipeline.modules()[9].module_name)
    pipeline.remove_module(10)

print('Pipeline modules:')
for module in pipeline.modules():
    print(module.module_num, module.module_name)

Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
Remove module:  OverlayOutlines
Remove module:  SaveImages
Remove module:  SaveImages
Remove module:  SaveImages
Pipeline modules:
1 RescaleIntensity
2 RescaleIntensity
3 CorrectIlluminationCalculate
4 CorrectIlluminationCalculate
5 CorrectIlluminationApply
6 IdentifyPrimaryObjects
7 MaskImage
8 IdentifyPrimaryObjects
9 IdentifyPrimaryObjects
10 MeasureColocalization
11 ExportToSpreadsheet


### Run Cell Profiler Pipeline on the OMERO Images
First we create a dataframe to store all the results.

In [6]:
image_df = pandas.DataFrame()

Now let's get a grasp of the Dataset to analize now. You need to provide the dataset ID to analyze

In [21]:
channel_names = [
    'HA',
    'Pc',
]


dataset_id = int(input("Dataset id: "))
image_ids = ezomero.get_image_ids(conn, dataset=dataset_id, across_groups=False)

for image_id in image_ids[:5]:

    image, image_data = ezomero.get_image(conn, image_id)
    print(f"Processing image: {image.getName()}...")
    
    # For each Image in OMERO, we copy pipeline and inject image modules
    pipeline_copy = pipeline.copy()

    for channel, channel_name in enumerate(channel_names):

        inject_image_module = InjectImage(channel_name, np.squeeze(image_data[...,channel]))
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)

    m = pipeline_copy.run()
    
    print("Analysis done")

    # Results obtained as CSV from Cell Profiler
    image_csv = pandas.read_csv(new_output_directory + '/Image.csv')
    image_csv.insert(0, 'Image Name', image.getName())
    image_csv.insert(0, 'Image ID', image.getId())
    image_df = image_df.append(image_csv)
    
print("All images done")

Processing image: exp-20210305_sgs3-Gal4_UAS-Trr_Pc-Red_Ha-Green_DNA-DAPI-03-ApoTome RAW Convert-17.czi...
Analysis done


## Exploring the output
The output is stored in memory as a pandas dataframe. Let's have a quick look at it.

In [21]:
image_df.head()

,Correlation_Costes_CorrHA_CorrPc,Correlation_Costes_CorrPc_CorrHA,Correlation_Manders_CorrHA_CorrPc,Correlation_Manders_CorrPc_CorrHA,Count_Mask_nonStrong_Pc,Count_PcBands_strong,Count_PcBands_weak,ExecutionTime_01InjectImage,ExecutionTime_02InjectImage,ExecutionTime_03RescaleIntensity,...,Threshold_OrigThreshold_PcBands_strong,Threshold_OrigThreshold_PcBands_weak,Threshold_SumOfEntropies_Mask_nonStrong_Pc,Threshold_SumOfEntropies_PcBands_strong,Threshold_SumOfEntropies_PcBands_weak,Threshold_WeightedVariance_Mask_nonStrong_Pc,Threshold_WeightedVariance_PcBands_strong,Threshold_WeightedVariance_PcBands_weak,Image Name,Image ID
0,0.932581,0.812543,0.551168,0.672991,2.0,30.0,196.0,0.00,0.00,0.02,...,0.4,0.1,0.0,-10.910727,-12.295348,2.933796,2.671294,1.748441,exp-20210305_sgs3-Gal4_UAS-Trr_Pc-Red_Ha-Green...,38
0,0.937823,0.284427,0.193049,0.575878,1.0,22.0,283.0,0.02,0.00,0.02,...,0.4,0.1,0.0,-10.267540,-12.116251,1.507048,1.341739,1.368191,exp-20210305_sgs3-Gal4_UAS-Psc_Pc-Red_Ha-Green...,39
0,0.600197,0.137475,0.696986,0.186556,1.0,0.0,0.0,0.02,0.01,0.02,...,0.4,0.1,0.0,0.000000,0.000000,0.862746,0.862746,0.862746,exp-20210305_sgs3-Gal4_UAS-neg_Pc-Red_Ha-Green...,40
0,0.940885,0.662307,0.527255,0.926744,1.0,14.0,225.0,0.01,0.01,0.03,...,0.4,0.1,0.0,-10.612316,-11.857965,1.430076,1.265770,1.113389,exp-20210305_sgs3-Gal4_UAS-Psc_Pc-Red_Ha-Green...,41
0,0.693176,0.193549,0.801309,0.178604,1.0,0.0,0.0,0.00,0.00,0.03,...,0.4,0.1,0.0,0.000000,0.000000,0.889652,0.889652,0.889652,exp-20210305_sgs3-Gal4_UAS-neg_Pc-Red_Ha-Green...,42


### Push Results back to OMERO

In [16]:
file_to_upload = str(os.path.join(new_output_directory, 'Image.csv'))
image_df.to_csv(file_to_upload, index=False)

# create the original file and file annotation (uploads the file etc.)
namespace = "cellprofiler.analysis"
print("\nCreating an OriginalFile and FileAnnotation")
file_ann = conn.createFileAnnfromLocalFile(file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
print("Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(),
    ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize())
dataset.linkAnnotation(file_ann)


Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 98408 , Image.csv Size: 16787


<FileAnnotationWrapper id=98408>

#### Close the connection to the OMERO server

In [8]:
conn.close()